## GDAL with NetCDF and earthaccess

In [ ]:
import numpy as np
from common import earthaccess_args
from osgeo import gdal

In [ ]:
gdal.UseExceptions()

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
bucket = earthaccess_args[dataset]["bucket"]
folder = earthaccess_args[dataset]["folder"]
variable = earthaccess_args[dataset]["variable"]
scale = earthaccess_args[dataset]["scale"]
daac = earthaccess_args[dataset]["daac"]

In [ ]:
def configure_auth():
    import earthaccess

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials(daac)
    gdal.SetConfigOption("AWS_REGION", "us-west-2")
    gdal.SetConfigOption("AWS_SECRET_ACCESS_KEY", s3_credentials["secretAccessKey"])
    gdal.SetConfigOption("AWS_ACCESS_KEY_ID", s3_credentials["accessKeyId"])
    gdal.SetConfigOption("AWS_SESSION_TOKEN", s3_credentials["sessionToken"])

In [ ]:
def load_data():
    src = f"NETCDF:/vsis3/{bucket}/{folder}/{filename}:{variable}"
    ds = gdal.Open(src)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray().astype("float32", casting="unsafe")
    mask = band.GetMaskBand().ReadAsArray()
    ma = np.ma.masked_array(arr, np.logical_not(mask), fill_value=np.nan)
    if scale:
        np.multiply(ma, band.GetScale(), out=ma, casting="unsafe")
        np.add(ma, band.GetOffset(), out=ma, casting="unsafe")
    return ma.filled(fill_value=np.nan)

In [ ]:
if __name__ == "__main__":
    configure_auth()
    da = load_data()